# MsCA 31009 - Machine Learning and Predictive Analytics

## Project - Toxic Comment Classification

## Import files and libraries.

In [ ]:
#!pip3 install autocorrect

In [4]:
import pandas as pd
import numpy as np

import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from autocorrect import spell

from imblearn.over_sampling import SMOTE

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC

import re

[nltk_data] Downloading package punkt to /home/targoon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/targoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Download train data.**

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1hcoewV5fpD0kx8ysZsZi8EnSjxIgC0lp'
!unzip -o 'uc?export=download&id=1hcoewV5fpD0kx8ysZsZi8EnSjxIgC0lp'

--2018-11-03 10:02:36--  https://drive.google.com/uc?export=download&id=1hcoewV5fpD0kx8ysZsZi8EnSjxIgC0lp
Resolving drive.google.com (drive.google.com)... 172.217.4.206, 2607:f8b0:4009:807::200e
Connecting to drive.google.com (drive.google.com)|172.217.4.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6ua373f1v9qobh534h6f5ape3ahf82pm/1541253600000/00285997938321528797/*/1hcoewV5fpD0kx8ysZsZi8EnSjxIgC0lp?e=download [following]
--2018-11-03 10:02:39--  https://doc-00-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6ua373f1v9qobh534h6f5ape3ahf82pm/1541253600000/00285997938321528797/*/1hcoewV5fpD0kx8ysZsZi8EnSjxIgC0lp?e=download
Resolving doc-00-4c-docs.googleusercontent.com (doc-00-4c-docs.googleusercontent.com)... 172.217.5.1, 2607:f8b0:4009:806::2001
Connecting to doc-00-4c-docs.googleusercontent.com (doc-00-4c-docs.googl

In [4]:
toxic = pd.read_csv('train.csv')

## Data Preprocessing

### Text Cleaning

In [4]:
toxic.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


**Remove ID column.**

In [5]:
toxic.drop(['id'], axis=1, inplace=True)

**Remove non-alphabet characters**

In [6]:
toxic['comment_text'] = [re.sub('[^A-Za-z]', ' ', i).lower() for i in toxic['comment_text']]

**Tokenization**

In [7]:
toxic['comment_text_tokenize'] = [word_tokenize(i) for i in toxic['comment_text']]

KeyboardInterrupt: 

In [ ]:
toxic.head()

**Standardize contraction**

In [13]:
for i in range(6):
    print(confusion_matrix_test_cv_all[i])def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"cant", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

**Stemming**

In [14]:
stemmer = SnowballStemmer('english')
stentence_placeholder = []
for sentence in toxic.loc[:,'comment_text_tokenize']:
    sentence_stemmed = [stemmer.stem(clean_text(word)) for word in sentence]
    stentence_placeholder.append(sentence_stemmed)
toxic['comment_text_tokenize_stemmed'] = stentence_placeholder

**Stopwords Removal**

In [15]:
stentence_placeholder = []
for sentence in toxic.loc[:,'comment_text_tokenize_stemmed']:
    sentence_clean = [word for word in sentence if word not in stopwords.words('english')]
    stentence_placeholder.append(sentence_clean)
toxic['comment_text_clean'] = stentence_placeholder
toxic['comment_text_clean'] = [' '.join(i) for i in toxic['comment_text_clean']]

In [16]:
toxic

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_tokenize,comment_text_tokenize_stemmed,comment_text_clean
0,explanation why the edits made under my userna...,0,0,0,0,0,0,"[explanation, why, the, edits, made, under, my...","[explan, whi, the, edit, made, under, my, user...",explan whi edit made usernam hardcor metallica...
1,d aww he matches this background colour i m s...,0,0,0,0,0,0,"[d, aww, he, matches, this, background, colour...","[d, aww, he, match, this, background, colour, ...",aww match background colour seem stuck thank t...
2,hey man i m really not trying to edit war it...,0,0,0,0,0,0,"[hey, man, i, m, really, not, trying, to, edit...","[hey, man, i, m, realli, not, tri, to, edit, w...",hey man realli tri edit war guy constant remov...
3,more i can t make any real suggestions on im...,0,0,0,0,0,0,"[more, i, can, t, make, any, real, suggestions...","[more, i, can, t, make, ani, real, suggest, on...",make ani real suggest improv wonder section st...
4,you sir are my hero any chance you remember...,0,0,0,0,0,0,"[you, sir, are, my, hero, any, chance, you, re...","[you, sir, are, my, hero, ani, chanc, you, rem...",sir hero ani chanc rememb page
5,congratulations from me as well use the to...,0,0,0,0,0,0,"[congratulations, from, me, as, well, use, the...","[congratul, from, me, as, well, use, the, tool...",congratul well use tool well talk
6,cocksucker before you piss around on my work,1,1,1,0,1,0,"[cocksucker, before, you, piss, around, on, my...","[cocksuck, befor, you, piss, around, on, my, w...",cocksuck befor piss around work
7,your vandalism to the matt shirvington article...,0,0,0,0,0,0,"[your, vandalism, to, the, matt, shirvington, ...","[your, vandal, to, the, matt, shirvington, art...",vandal matt shirvington articl revert pleas ban
8,sorry if the word nonsense was offensive to ...,0,0,0,0,0,0,"[sorry, if, the, word, nonsense, was, offensiv...","[sorri, if, the, word, nonsens, was, offens, t...",sorri word nonsens offens anyway intend write ...
9,alignment on this subject and which are contra...,0,0,0,0,0,0,"[alignment, on, this, subject, and, which, are...","[align, on, this, subject, and, which, are, co...",align subject contrari dulithgow


In [17]:
toxic.to_csv('train_cleaned.csv', index=False)

### Create feature spaces

In [3]:
toxic = pd.read_csv('train_cleaned.csv')

**Drop NA**

In [25]:
toxic.describe(include='all')

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_tokenize,comment_text_tokenize_stemmed,comment_text_clean
count,159521,159521.000000,159521.000000,159521.000000,159521.000000,159521.000000,159521.000000,159521,159521,159521
unique,159255,NaN,NaN,NaN,NaN,NaN,NaN,158206,158181,157648
top,jun utc,NaN,NaN,NaN,NaN,NaN,NaN,['january'],['januari'],thank experi wikipedia test work revert remov ...
freq,11,NaN,NaN,NaN,NaN,NaN,NaN,21,21,22
mean,NaN,0.095875,0.009999,0.052965,0.002996,0.049379,0.008808,NaN,NaN,NaN
std,NaN,0.294420,0.099493,0.223964,0.054658,0.216659,0.093435,NaN,NaN,NaN
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
50%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
75%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [5]:
toxic.dropna(axis=0, inplace=True)

**Split Train and Test**

In [6]:
x_train, x_test, y_train, y_test = train_test_split(toxic.loc[:,'comment_text_clean'], toxic.iloc[:,1:7], test_size = .3, random_state = 43)

In [7]:
x_train.head()

21524    thank note worri wait period get permiss owner...
56229    page need massiv edit initi section befor hit ...
93765                                       okaaaaaay test
87443    apologis make remark sidaway return perhap cou...
73667    newspap headlin newspap headlin blank adult sw...
Name: comment_text_clean, dtype: object

In [8]:
x_train.shape

(111664,)

In [9]:
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
21524,0,0,0,0,0,0
56229,0,0,0,0,0,0
93765,0,0,0,0,0,0
87443,0,0,0,0,0,0
73667,0,0,0,0,0,0


**Create feature spaces**

In [10]:
#Count Vectors as features

count_vect = CountVectorizer(max_features=5000)
count_vect.fit(x_train)
x_train_cv = count_vect.transform(x_train)
x_test_cv = count_vect.transform(x_test)

#TF-IDF Vectors as features

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(x_train)
x_train_tfidf =  tfidf_vect.transform(x_train)
x_test_tfidf =  tfidf_vect.transform(x_test)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(x_train)
x_train_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
x_test_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

In [11]:
feature_name_cv = count_vect.get_feature_names()
feature_name_tfidf = tfidf_vect.get_feature_names()
feature_name_ngram = tfidf_vect_ngram.get_feature_names()

In [12]:
print(feature_name_tfidf)

['aa', 'aaron', 'ab', 'abandon', 'abbrevi', 'abc', 'abid', 'abil', 'abl', 'abort', 'abov', 'abraham', 'abroad', 'absenc', 'absent', 'absolut', 'abstract', 'absurd', 'abund', 'abus', 'ac', 'academ', 'academi', 'acceler', 'accent', 'accept', 'access', 'accid', 'accident', 'accommod', 'accompani', 'accomplish', 'accord', 'account', 'accur', 'accuraci', 'accus', 'ace', 'achiev', 'acid', 'acknowledg', 'acquir', 'acronym', 'across', 'act', 'action', 'activ', 'activist', 'actor', 'actress', 'actual', 'ad', 'adam', 'adapt', 'add', 'addict', 'addit', 'address', 'adequ', 'adher', 'adject', 'adjust', 'admin', 'administ', 'administr', 'adminship', 'admir', 'admiss', 'admit', 'adolf', 'adopt', 'adress', 'adult', 'advanc', 'advantag', 'adventur', 'advert', 'advertis', 'advic', 'advis', 'advoc', 'advocaci', 'ae', 'aesthet', 'afc', 'afd', 'affair', 'affect', 'affili', 'affirm', 'afford', 'afghan', 'afghanistan', 'aforement', 'afraid', 'africa', 'african', 'afternoon', 'afterward', 'age', 'agenc', 'age

### Oversampling (SMOTE)

In [13]:
x_train_cv_os_all = []
y_train_cv_os_all = []

x_train_tfidf_os_all = []
y_train_tfidf_os_all = []

x_train_ngram_os_all = []
y_train_ngram_os_all = []


for i in range(6):
    sm_cv = SMOTE(random_state=40)
    x_train_cv_os, y_train_cv_os = sm_cv.fit_resample(x_train_cv, y_train.iloc[:,i])
    x_train_cv_os_all.append(x_train_cv_os)
    y_train_cv_os_all.append(y_train_cv_os)
    
    sm_tfidf = SMOTE(random_state=40)
    x_train_tfidf_os, y_train_tfidf_os = sm_tfidf.fit_resample(x_train_tfidf, y_train.iloc[:,i])
    x_train_tfidf_os_all.append(x_train_tfidf_os)
    y_train_tfidf_os_all.append(y_train_tfidf_os)
    
    sm_ngram = SMOTE(random_state=40)
    x_train_ngram_os, y_train_ngram_os = sm_ngram.fit_resample(x_train_tfidf_ngram, y_train.iloc[:,i])
    x_train_ngram_os_all.append(x_train_ngram_os)
    y_train_ngram_os_all.append(y_train_ngram_os)

In [14]:
x_train_y_train_all = [x_train_cv_os_all, y_train_cv_os_all, x_train_tfidf_os_all, y_train_tfidf_os_all, x_train_ngram_os_all, y_train_ngram_os_all]

In [49]:
x_test_y_test_all = [x_test_cv, y_test, x_test_tfidf, y_test, x_test_tfidf_ngram, y_test]

In [15]:
for i in x_train_cv_os_all:
    print(i.shape)

(201698, 5000)
(221074, 5000)
(211334, 5000)
(222628, 5000)
(212120, 5000)
(221320, 5000)


In [35]:
import pickle

# where do I want to store this file?
# Open the file to save as pkl file
train_data_path = 'train_data_array.pkl'
train_data_path_pkl = open(train_data_path, 'wb')
pickle.dump(x_train_y_train_all, train_data_path_pkl)

# Close the pickle instances
train_data_path_pkl.close()

In [50]:
test_data_path = 'test_data_array.pkl'
test_data_path_pkl = open(test_data_path, 'wb')
pickle.dump(x_test_y_test_all, test_data_path_pkl)

## Load Feature Matrices

In [1]:
import pickle
x_train_y_train_all_load = pickle.load(open('train_data_array.pkl', 'rb'))
x_test_y_test_all_load = pickle.load(open('test_data_array.pkl', 'rb'))

In [2]:
x_train_cv_os_all = x_train_y_train_all_load[0]
y_train_cv_os_all = x_train_y_train_all_load[1]

x_train_tfidf_os_all = x_train_y_train_all_load[2]
y_train_tfidf_os_all = x_train_y_train_all_load[3]

x_train_ngram_os_all = x_train_y_train_all_load[4]
y_train_ngram_os_all = x_train_y_train_all_load[5]

In [3]:
x_test_cv = x_test_y_test_all_load[0]
x_test_tfidf = x_test_y_test_all_load[2]
x_test_tfidf_ngram = x_test_y_test_all_load[4]
y_test = x_test_y_test_all_load[1]

y_test = [np.array(y_test.iloc[:,i]).reshape(-1,1) for i in range(6)]

## Model Selection

### Logistic Regression

#### Count Vector Feature Space

In [41]:
class toxicmodel:
    def __init__(self, x_train, y_train, x_test, y_test, n = 6):
        self.n = n
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
        self.best_params = []
        self.best_estimator = []
        
        self.y_predict_train = []
        self.y_predict_test = []
        self.y_predict_proba_train = []
        self.y_predict_proba_test = []

        self.acc_score_train = []
        self.acc_score_test = []

        self.roc_auc_score_train = []
        self.roc_auc_score_test = []

        self.f1_score_train = []
        self.f1_score_test = []

        self.confusion_matrix_train = []
        self.confusion_matrix_test = []

        self.classification_report_train = []
        self.classification_report_test = []

    
    def trainmodel(self, model_name, hyper_param_grid):
        for i in range(self.n):
            grid_search_model = GridSearchCV(model_name, hyper_param_grid, scoring = 'f1', cv = 5,refit = True, n_jobs=-1, verbose = 5)
            grid_search_model.fit(self.x_train[i], self.y_train[i])
            self.best_params.append(grid_search_model.best_params_)
            self.best_estimator.append(grid_search_model.best_estimator_)
    
    
    def predictmodel(self):
        for i in range(self.n):
            
            y_predict_train = self.best_estimator[i].predict(self.x_train[i])
            y_predict_test = self.best_estimator[i].predict(self.x_test)
             
            #y_predict_proba_train = self.best_estimator[i].predict_proba(self.x_train[i])[:,1]
            #y_predict_proba_test = self.best_estimator[i].predict_proba(self.x_test)[:,1]
            

            #self.y_predict_train.append(y_predict_train)
            #self.y_predict_test.append(y_predict_test)
            
            #self.y_predict_proba_train.append(y_predict_proba_train)
            #self.y_predict_proba_test.append(y_predict_proba_test)

            #self.roc_auc_score_train.append(roc_auc_score(self.y_train[i], y_predict_proba_train))
            #self.roc_auc_score_test.append(roc_auc_score(self.y_test[i], y_predict_proba_test))
            
            self.acc_score_train.append(accuracy_score(self.y_train[i], y_predict_train))
            self.acc_score_test.append(accuracy_score(self.y_test[i], y_predict_test))
            
            self.f1_score_train.append(f1_score(self.y_train[i], y_predict_train))
            self.f1_score_test.append(f1_score(self.y_test[i], y_predict_test))

            self.confusion_matrix_train.append(confusion_matrix(self.y_train[i], y_predict_train))
            self.confusion_matrix_test.append(confusion_matrix(self.y_test[i], y_predict_test))

            self.classification_report_train.append(classification_report(self.y_train[i], y_predict_train))
            self.classification_report_test.append(classification_report(self.y_test[i], y_predict_test))

In [8]:
if __name__ == '__main__':
    log_toxic = toxicmodel(x_train_cv_os_all, y_train_cv_os_all, x_test_cv, y_test)
    log_toxic.trainmodel(LogisticRegression(), {'random_state':[0]})
    log_toxic.predictmodel()
    

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   11.7s remaining:   17.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.4s finished
/home/targoon/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/targoon/.local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   11.6s remaining:   17.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.5s finished
/home/targoon/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/targoon/.local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   13.2s remaining:   19.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.8s finished
/home/targoon/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/targoon/.local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   11.0s remaining:   16.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.5s finished
/home/targoon/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/targoon/.local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   13.2s remaining:   19.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.8s finished
/home/targoon/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/targoon/.local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/targoon/.local/lib/python3.6/site-packages/sklearn/linear_model/base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/home/targoon/.local/lib/python3.6/site-packages/sklearn/linear_model/base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [ ]:
svml_toxic = toxicmodel(x_train_cv_os_all, y_train_cv_os_all, x_test_cv, y_test, n = 1)
svml_toxic.trainmodel(SVC(kernel='linear', probability=True), {'C':np.arange(0.01,0.1,0.02)})
svml_toxic.predictmodel()

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
svml_toxic = toxicmodel(n=1, x_train = x_train_cv_os_all,y_train=y_train_cv_os_all,x_test=x_test_cv, y_test=y_test)
svml_toxic.trainmodel(RandomForestClassifier(), {'n_estimators':[500, 750, 1000],'max_features':[10, 25, 40, 65],'random_state':[0], 'max_depth':[4,6,8]})
svml_toxic.predictmodel()

In [ ]:
svml_toxic.best_estimator

In [ ]:
svml_toxic.acc_score_test

In [ ]:
svml_toxic.acc_score_train

In [ ]:
svml_toxic.acc_score_train

### Neural Network

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils

In [12]:
model = Sequential()
model.add(Dense(512, input_shape=(5000,)))
model.add(Activation('relu')) 
 
# Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dropout(0.2))   

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               2560512   
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 1026      
__________

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
y_train = np_utils.to_categorical(y_train_cv_os_all[0], 2)
y_test = np_utils.to_categorical(y_test[0], 2)

In [22]:
history = model.fit(x_train_cv_os_all[0], y_train,
          batch_size=128, epochs=20,
          verbose=1,
          validation_data=(x_test_cv, y_test))

Train on 201698 samples, validate on 47857 samples
Epoch 1/20
201698/201698 [==============================] - 178s 883us/step - loss: 0.1671 - val_loss: 0.1771
Epoch 2/20
201698/201698 [==============================] - 161s 799us/step - loss: 0.0924 - val_loss: 0.2066
Epoch 3/20
180736/201698 [=========================>....] - ETA: 16s - loss: 0.0551

KeyboardInterrupt: 